In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta
import requests
import io

In [ ]:
class SP500NeuralNetwork:
    def __init__(self, lookback_days=30):
        """
        Initialize the neural network for S&P 500 stocks prediction
        
        Parameters:
        lookback_days (int): Number of previous days to use for prediction
        """
        self.lookback_days = lookback_days
        
    def get_sp500_tickers(self):
        """Get current S&P 500 stock tickers"""
        # Get S&P 500 tickers from Wikipedia
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        sp500_table = tables[0]
        return sp500_table['Symbol'].tolist()
    
    def get_stock_data(self, tickers):
        """
        Fetch historical data for all S&P 500 stocks
        
        Parameters:
        tickers (list): List of stock tickers
        """
        end_date = datetime.now()
        start_date = end_date - timedelta(days=5*365)  # 10 years of data
        
        all_stocks_data = {}
        print("Fetching data for all S&P 500 stocks...")
        
        for i, ticker in enumerate(tickers):
            try:
                stock = yf.Ticker(ticker)
                data = stock.history(start=start_date, end=end_date)
                if not data.empty:
                    all_stocks_data[ticker] = data['Close']
                if (i + 1) % 50 == 0:
                    print(f"Processed {i + 1} stocks")
            except Exception as e:
                print(f"Error fetching data for {ticker}: {str(e)}")
                continue
                
        return pd.DataFrame(all_stocks_data)
    
    def prepare_data(self, stock_prices):
        """
        Prepare data for neural network training
        Returns sequences of lookback_days and their corresponding next day movements
        """
        X, Y = [], []
        
        # Process each stock
        for column in stock_prices.columns:
            prices = stock_prices[column].dropna().values
            if len(prices) <= self.lookback_days:
                continue
                
            # Create sequences for this stock
            for i in range(len(prices) - self.lookback_days):
                sequence = prices[i:(i + self.lookback_days)]
                next_price = prices[i + self.lookback_days]
                
                # Only add if we have valid data
                if not np.any(np.isnan(sequence)) and not np.isnan(next_price):
                    X.append(sequence)
                    Y.append(1 if next_price > prices[i + self.lookback_days - 1] else 0)
        
        X = np.array(X)
        Y = np.array(Y)
        
        if len(X) == 0:
            raise ValueError("No valid sequences found in the data")
        
        # Normalize each sequence
        X = (X - np.mean(X, axis=1, keepdims=True)) / np.std(X, axis=1, keepdims=True)
        
        # Replace any remaining NaN values
        X = np.nan_to_num(X)
        
        # Split into train and dev sets
        split = int(0.8 * len(X))
        X_train, X_dev = X[:split].T, X[split:].T
        Y_train, Y_dev = Y[:split], Y[split:]
        
        return X_train, Y_train, X_dev, Y_dev
    
    def init_params(self):
        """Initialize neural network parameters"""
        W1 = np.random.rand(20, self.lookback_days) - 0.5  # Increased hidden layer size
        b1 = np.random.rand(20, 1) - 0.5
        W2 = np.random.rand(2, 20) - 0.5
        b2 = np.random.rand(2, 1) - 0.5
        return W1, b1, W2, b2
    
    def ReLU(self, Z):
        """ReLU activation function"""
        return np.maximum(Z, 0)
    
    def ReLU_deriv(self, Z):
        """Derivative of ReLU function"""
        return Z > 0
    
    def softmax(self, Z):
        """Softmax activation function"""
        exp = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp / np.sum(exp, axis=0, keepdims=True)
    
    def forward_prop(self, W1, b1, W2, b2, X):
        """Forward propagation step"""
        Z1 = W1.dot(X) + b1
        A1 = self.ReLU(Z1)
        Z2 = W2.dot(A1) + b2
        A2 = self.softmax(Z2)
        return Z1, A1, Z2, A2
    
    def one_hot(self, Y):
        """Convert labels to one-hot encoding"""
        one_hot_Y = np.zeros((2, Y.size))
        one_hot_Y[Y, np.arange(Y.size)] = 1
        return one_hot_Y
    
    def backward_prop(self, Z1, A1, Z2, A2, W2, X, Y, m):
        """Backward propagation step"""
        one_hot_Y = self.one_hot(Y)
        dZ2 = A2 - one_hot_Y
        dW2 = 1 / m * dZ2.dot(A1.T)
        db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = W2.T.dot(dZ2) * self.ReLU_deriv(Z1)
        dW1 = 1 / m * dZ1.dot(X.T)
        db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
        return dW1, db1, dW2, db2
    
    def update_params(self, W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
        """Update parameters using gradient descent"""
        W1 = W1 - alpha * dW1
        b1 = b1 - alpha * db1
        W2 = W2 - alpha * dW2
        b2 = b2 - alpha * db2
        return W1, b1, W2, b2
    
    def get_predictions(self, A2):
        """Get predictions from network output"""
        return np.argmax(A2, 0)
    
    def get_accuracy(self, predictions, Y):
        """Calculate prediction accuracy"""
        return np.sum(predictions == Y) / Y.size
    
    def train(self, X, Y, alpha=0.01, iterations=10000):
        """Train the neural network"""
        W1, b1, W2, b2 = self.init_params()
        m = X.shape[1]
        
        for i in range(iterations):
            Z1, A1, Z2, A2 = self.forward_prop(W1, b1, W2, b2, X)
            dW1, db1, dW2, db2 = self.backward_prop(Z1, A1, Z2, A2, W2, X, Y, m)
            W1, b1, W2, b2 = self.update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
            
            if i % 100 == 0:
                predictions = self.get_predictions(A2)
                accuracy = self.get_accuracy(predictions, Y)
                print(f"Iteration: {i}, Training Accuracy: {accuracy:.4f}")
                
        return W1, b1, W2, b2
    
    def predict(self, X, W1, b1, W2, b2):
        """Make predictions using trained parameters"""
        _, _, _, A2 = self.forward_prop(W1, b1, W2, b2, X)
        return self.get_predictions(A2)

def main():
    # Initialize neural network
    nn = SP500NeuralNetwork(lookback_days=30)
    
    # Get S&P 500 tickers
    print("Fetching S&P 500 tickers...")
    tickers = nn.get_sp500_tickers()
    
    # Get historical data for all stocks
    stock_prices = nn.get_stock_data(tickers)
    
    # Prepare data
    print("\nPreparing data...")
    X_train, Y_train, X_dev, Y_dev = nn.prepare_data(stock_prices)
    
    print(f"\nTraining data shape: {X_train.shape}")
    print(f"Development data shape: {X_dev.shape}")
    
    # Train the network
    print("\nTraining neural network...")
    W1, b1, W2, b2 = nn.train(X_train, Y_train)
    
    # Test on development set
    dev_predictions = nn.predict(X_dev, W1, b1, W2, b2)
    accuracy = nn.get_accuracy(dev_predictions, Y_dev)
    print(f"\nDevelopment Set Accuracy: {accuracy:.4f}")
    
    # Save the model parameters
    np.savez('sp500_model.npz', W1=W1, b1=b1, W2=W2, b2=b2)
    print("\nModel parameters saved to 'sp500_model.npz'")

if __name__ == "__main__":
    main()

In [ ]:
# # Then run:
# nn = SP500NeuralNetwork(lookback_days=30)
# tickers = nn.get_sp500_tickers()
# stock_prices = nn.get_stock_data(tickers)
# X_train, Y_train, X_dev, Y_dev = nn.prepare_data(stock_prices)
# W1, b1, W2, b2 = nn.train(X_train, Y_train)

In [ ]:
class SP500NeuralNetwork:
    def __init__(self, lookback_days=30):
        """Initialize the neural network for S&P 500 stock predictions."""
        self.lookback_days = lookback_days

    def get_sp500_tickers(self):
        """Fetch the current list of S&P 500 tickers."""
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        sp500_table = tables[0]
        return sp500_table['Symbol'].tolist()

    def get_stock_data(self, tickers):
        """Fetch historical stock data for the given tickers."""
        end_date = datetime.now()
        start_date = end_date - timedelta(days=5 * 365)
        all_stocks_data = {}

        print("Fetching stock data for all S&P 500 companies...")
        for i, ticker in enumerate(tickers):
            try:
                stock = yf.Ticker(ticker)
                data = stock.history(start=start_date, end=end_date)
                if not data.empty:
                    all_stocks_data[ticker] = data['Close']
                if (i + 1) % 50 == 0:
                    print(f"Processed {i + 1} stocks...")
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
                continue
        return pd.DataFrame(all_stocks_data)

    def prepare_data(self, stock_prices):
        """Prepare data for neural network training."""
        X, Y = [], []

        for column in stock_prices.columns:
            prices = stock_prices[column].dropna().values
            if len(prices) <= self.lookback_days:
                continue
            for i in range(len(prices) - self.lookback_days):
                sequence = prices[i:(i + self.lookback_days)]
                next_price = prices[i + self.lookback_days]
                X.append(sequence)
                Y.append(1 if next_price > prices[i + self.lookback_days - 1] else 0)

        X = np.array(X)
        Y = np.array(Y)

        if len(X) == 0:
            raise ValueError("No valid sequences found in the data")

        X = (X - np.mean(X, axis=1, keepdims=True)) / np.std(X, axis=1, keepdims=True)
        X = np.nan_to_num(X)

        split = int(0.8 * len(X))
        X_train, X_dev = X[:split].T, X[split:].T
        Y_train, Y_dev = Y[:split], Y[split:]
        return X_train, Y_train, X_dev, Y_dev

    def init_params(self, layer_sizes):
        """Initialize neural network parameters."""
        np.random.seed(42)
        params = {}
        for i in range(1, len(layer_sizes)):
            params[f"W{i}"] = np.random.randn(layer_sizes[i], layer_sizes[i - 1]) * np.sqrt(2. / layer_sizes[i - 1])
            params[f"b{i}"] = np.zeros((layer_sizes[i], 1))
        return params

    def ReLU(self, Z):
        return np.maximum(Z, 0)

    def ReLU_deriv(self, Z):
        return Z > 0

    def softmax(self, Z):
        exp = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp / np.sum(exp, axis=0, keepdims=True)

    def forward_prop(self, X, params):
        caches = {"A0": X}
        L = len(params) // 2

        for i in range(1, L):
            Z = params[f"W{i}"].dot(caches[f"A{i-1}"]) + params[f"b{i}"]
            A = self.ReLU(Z)
            caches[f"Z{i}"] = Z
            caches[f"A{i}"] = A

        ZL = params[f"W{L}"].dot(caches[f"A{L-1}"]) + params[f"b{L}"]
        AL = self.softmax(ZL)
        caches[f"Z{L}"] = ZL
        caches[f"A{L}"] = AL
        return AL, caches

    def backward_prop(self, Y, caches, params, reg_lambda):
        m = Y.size
        grads = {}
        L = len(params) // 2
        A_last = caches[f"A{L}"]
        one_hot_Y = np.zeros((A_last.shape[0], m))
        one_hot_Y[Y, np.arange(m)] = 1

        dZL = A_last - one_hot_Y
        grads[f"dZ{L}"] = dZL
        grads[f"dW{L}"] = 1 / m * dZL.dot(caches[f"A{L-1}"].T) + (reg_lambda / m) * params[f"W{L}"]
        grads[f"db{L}"] = 1 / m * np.sum(dZL, axis=1, keepdims=True)

        for i in range(L - 1, 0, -1):
            dZ = params[f"W{i+1}"].T.dot(grads[f"dZ{i+1}"]) * self.ReLU_deriv(caches[f"Z{i}"])
            grads[f"dZ{i}"] = dZ
            grads[f"dW{i}"] = 1 / m * dZ.dot(caches[f"A{i-1}"].T) + (reg_lambda / m) * params[f"W{i}"]
            grads[f"db{i}"] = 1 / m * np.sum(dZ, axis=1, keepdims=True)

        return grads

    def update_params(self, params, grads, alpha):
        for key in params.keys():
            params[key] -= alpha * grads[f"d{key}"]
        return params

    def train(self, X, Y, layer_sizes, alpha=0.01, iterations=5000, reg_lambda=0.01):
        params = self.init_params(layer_sizes)
        m = X.shape[1]

        for i in range(iterations):
            AL, caches = self.forward_prop(X, params)
            grads = self.backward_prop(Y, caches, params, reg_lambda)
            params = self.update_params(params, grads, alpha)

            if i % 100 == 0:
                predictions = np.argmax(AL, axis=0)
                accuracy = np.mean(predictions == Y)
                print(f"Iteration {i}, Accuracy: {accuracy:.4f}")

        return params

    def predict(self, X, params):
        AL, _ = self.forward_prop(X, params)
        return np.argmax(AL, axis=0)

def main():
    nn = SP500NeuralNetwork(lookback_days=30)
    tickers = nn.get_sp500_tickers()
    stock_prices = nn.get_stock_data(tickers)

    print("\nPreparing data...")
    X_train, Y_train, X_dev, Y_dev = nn.prepare_data(stock_prices)

    layer_sizes = [X_train.shape[0], 128, 64, 32, 2]
    print("\nTraining neural network...")
    params = nn.train(X_train, Y_train, layer_sizes, alpha=0.001, iterations=5000, reg_lambda=0.01)

    print("\nEvaluating on development set...")
    dev_predictions = nn.predict(X_dev, params)
    dev_accuracy = np.mean(dev_predictions == Y_dev)
    print(f"Development Set Accuracy: {dev_accuracy:.4f}")

if __name__ == "__main__":
    main()
